In [102]:
from sympy import cacheit, Symbol
from sympy.core.function import Function, UndefinedFunction, Application


class FOLConstant(Symbol):
    is_Constant = True


class Formula(Function):
    is_Predicate = False
    is_Quantifier = False

    # called when operation is not allowed (as in sympy library)
    def _noop(self, other=None):
        raise TypeError('First order logic term not allowed in this context.')

    # redefinition of and
    def __and__(self, other):
        """Overloading for & operator"""
        return FOLAnd(self, other)

    __rand__ = __and__

    # redefinition of or
    def __or__(self, other):
        """Overloading for |"""
        return FOLOr(self, other)

    __ror__ = __or__

    # redefinition of not
    def __invert__(self):
        """Overloading for ~"""
        return FOLNot(self)

    # redefinition of implies
    def __rshift__(self, other):
        """Overloading for >>"""
        return FOLImplies(self, other)

    # redefinition of is implied
    def __lshift__(self, other):
        """Overloading for <<"""
        return FOLImplies(other, self)

    __rrshift__ = __lshift__
    __rlshift__ = __rshift__

    # redefinition of is xor
    def __xor__(self, other):
        return FOLOr(FOLAnd(self, FOLNot(other)), FOLAnd(FOLNot(self), other))

    __rxor__ = __xor__

    __add__ = _noop
    __radd__ = _noop
    __sub__ = _noop
    __rsub__ = _noop
    __mul__ = _noop
    __rmul__ = _noop
    __pow__ = _noop
    __rpow__ = _noop
    __rdiv__ = _noop
    __truediv__ = _noop
    __div__ = _noop
    __rtruediv__ = _noop
    __mod__ = _noop
    __rmod__ = _noop
    _eval_power = _noop

    def _apply_not(self):
        raise NotImplementedError("Implementation Error")

    # negation normal form
    def to_nnf(self):
        raise NotImplementedError("Implementation Error")

    # skolem normal form
    def to_snf(self, skolem_function_symbol= None):
        from sympy.first_order_logic.algorithm import to_snf
        return to_snf(self, skolem_function_symbol)

    # prenex normal form
    def to_pnf(self):
        from sympy.first_order_logic.algorithm import to_pnf
        return to_pnf(self)

    def to_list_clauses(self):
        from sympy.first_order_logic.algorithm import to_list_clauses
        return to_list_clauses(self)

    def is_satisfiable(self, algorithm=None):
        f = self.to_nnf()
        f = f.to_pnf()
        f = f.to_snf()
        if algorithm is None:
            from sympy.first_order_logic.algorithm import Resolution
            algorithm = Resolution()
        c = f.to_list_clauses()
        return algorithm(*c)

"""
def func(self):
        return self.__class__
"""
class FOLLogicOperator(Formula):
    def to_nnf(self):
        # every argument of the formula must have its own to_nnf() method
        return self.func(*[a.to_nnf() for a in self.args])


class FOLAnd(FOLLogicOperator):
    nargs = 2

    # not(A and B) = not A or not B
    def _apply_not(self):
        return FOLOr(self.args[0]._apply_not(), self.args[1]._apply_not())
    
    #def _latex(self,printer,*args):
    #    return r'%s \bigwedge %s ' % tuple([printer._print(args[0],*args), printer._print(args[1],*args)])


class FOLOr(FOLLogicOperator):
    nargs = 2
    
    # not(A or B) = not A and not B
    def _apply_not(self):
        return FOLAnd(self.args[0]._apply_not(), self.args[1]._apply_not())
    
    #def _latex(self,printer,*args):
    #    return r'%s \bigvee %s ' % tuple([printer._print(args[0],*args), printer._print(args[1],*args)])



class FOLNot(FOLLogicOperator):
    nargs = 1

    # in NNF the logic operator not must only be in front of predicates
    def to_nnf(self):
        expr = self.args[0]
        if (expr.is_Predicate):
            return self

        res = self.args[0]._apply_not()
        return res.to_nnf()
    
    # not (not A) = A
    def _apply_not(self):
        return self.args[0]

    #def _latex(self,printer,*args):
    #    return r'\lnot %s ' % printer._print(args[0],*args)


class FOLImplies(FOLLogicOperator):
    nargs = 2

    # A -> B = not A or B
    def to_nnf(self):
        return FOLOr(self.args[0]._apply_not().to_nnf(), self.args[1].to_nnf())

    # not (A -> B) = A and not B
    def _apply_not(self):
        return FOLAnd(self.args[0], self.args[1]._apply_not())


class Predicate(Formula):
    is_Atom = True
    is_Predicate = True
    is_Equality = False

    @cacheit
    def __new__(cls, *args, **options):
        if cls is Predicate:
            options['bases'] = (AppliedPredicate,)
            # a predicate is considered as an UndefinedFunction
            res = UndefinedFunction(*args, **options)
            return res

        return super(Predicate, cls).__new__(cls, *args, **options)

    def to_nnf(self):
        return self

    def _apply_not(self):
        return FOLNot(self)

    


class FOLEquality(Predicate):
    is_Equality = True
    nargs = 2

class AppliedPredicate(Predicate):
    pass


class Quantifier(Formula):
    nargs = 1
    is_Quantifier = True
    is_Forall = False
    is_Exists = False
    _symbol = None

    def __new__(cls, *args, **options):
        if cls in (Forall, Exists):
            argslist = list(args)
            symbol = argslist[0] # variable which is not free
            argslist[0] = cls.name + symbol.name
            args = tuple(argslist)
            options['bases'] = (cls._getAppliedClass(),)
            res = UndefinedFunction(*args, **options)
            res._symbol = symbol
            return res

        return super(Quantifier, cls).__new__(cls, *args, **options)

    @property
    def symbol(self):
        return self._symbol

    @property
    def formula(self):
        return self.args[0]

    def to_nnf(self):
        return self.func(self.formula.to_nnf())


class Forall(Quantifier):
    is_Forall = True
    name = 'Forall_'
    
    # not forall = exist not
    def _apply_not(self):
        exists = Exists(self.symbol)
        return exists(self.formula._apply_not())

    @staticmethod
    def _getAppliedClass():
        return AppliedForall
    
    def _latex(self,printer,*args):
        return r'\forall %s (%s) ' % tuple([printer._print(self.symbol,*args), printer._print(self.formula,*args)])


class AppliedForall(Forall):
    pass


class Exists(Quantifier):
    is_Exists = True
    name = 'Exists_'

    # not exists = forall not
    def _apply_not(self):
        forall = Forall(self.symbol)
        return forall(self.formula._apply_not())

    @staticmethod
    def _getAppliedClass():
        return AppliedExists
    
    def _latex(self,printer,*args):
        return r'\exists %s (%s) ' % tuple([printer._print(self.symbol,*args), printer._print(self.formula,*args)])
    


class AppliedExists(Exists):
    pass

def grounding(formula, constants_set):
    # transform the formula in NNF
    formula = formula.to_nnf()
    # cercare se ci sono quantifiers nella formula
    i=0
    arg = formula.args[i]
    while not arg.is_Quantifier:
        if arg.func == Forall:
            # arg.symbol -> variable
            # arg.formula -> formula
            grounds = arg.formula.xreplace({arg.symbol: constants_set[0]})
            for constant in constants_set[1:]:
                temp = arg.formula.xreplace({arg.symbol: constant})
                grounds = FOLAnd(grounds, temp)
        elif arg.func == Exists:
            grounds = arg.formula.xreplace({arg.symbol: constants_set[0]})
            for constant in constants_set[1:]:
                temp = arg.formula.xreplace({arg.symbol: constant})
                grounds = FOLOr(grounds, temp)
        i += 1
        arg = formula.args[i]
        

class Clause(Application):
    _formula = None

    def __new__(cls, *args, **kwargs):
        args_list = list(args)
        if([] in args_list):
            # remove empty clause
            args_list.remove([])
        # if we have only one clause and it starts with a quatifier
        if len(args_list) == 1 and not args_list[0].is_Predicate and args_list[0].func is not FOLNot:
            from sympy.first_order_logic.algorithm import remove_quantifiers, formula_to_list
            f,_ = remove_quantifiers(args_list[0])
            # clause form
            args = formula_to_list(f, FOLOr)
        
        return super(Clause, cls).__new__(cls, *args, **kwargs)

    def substitute(self, sub):
        f = self.formula.xreplace(sub)
        return Clause(f)

    def factoring(self):
        from sympy.first_order_logic.algorithm import factoring
        return factoring(self)

    def create_resolvent(self, clause):
        from sympy.first_order_logic.algorithm import create_resolvent
        return create_resolvent(self, clause)

    def flatten(self):
        from sympy.first_order_logic.algorithm import flattening
        return flattening(self)

    @property
    def list(self):
        return list(self.args).copy()

    @property
    def formula(self):
        if self._formula is None:
            self._to_formula()

        return self._formula

    def _to_formula(self):
        f = None
        for a in self.args:
            if f is None:
                f = a
            else:
                f = FOLOr(f, a)

        for s in self._get_symbols(f).keys():
            f = Forall(s)(f)

        self._formula = f

    def lgg(self, clause):
        from sympy.first_order_logic.algorithm.least_general_generalization import lgg
        return lgg(self, clause)
    @classmethod
    def _get_symbols(cls, e):
        if e.func is Symbol and not e.is_Constant:
            return {e: True}

        dic = {}
        for a in e.args:
            dic.update(cls._get_symbols(a))
        return dic


In [103]:
from sympy.abc import x, y, z
f = Function('f')
P = Predicate('P')
#Exists(x) (Forall(y) (P(f(x),y) & z))

a=P(x)&x
a
#FOLNot(FOLAnd(x, y))
# Exists(x) (P(x,x))

NameError: name 'options' is not defined

FOLAnd(P(x), x)